<img src="img/heading_title.jpg" style="width: 100%; margin:0;" />

#

<img src="img/heading_overview.jpg" style="width: 100%; margin:0;" />

### 🖐 Wait here for further instructions

#

<img src="img/heading_domain.jpg" style="width: 100%; margin:0;" />

#

<img src="img/tab_install_pip.png" alt="tab" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
!sudo apt update && sudo apt install python3-pip

#

<img src="img/tab_install_hagrid.png" alt="tab" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
!pip install -U hagrid

#

<img src="img/tab_install_syft.png" alt="tab" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
!pip install --pre syft

#

<img src="img/tab_launch_domain.png" alt="tab" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
!hagrid launch to docker:80 --tag=latest --tail=false

#

<img src="img/tab_check_domain.png" alt="tab" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
!hagrid check --wait --silent

#

### 🖐 Raise your hand in Zoom and wait

#

<img src="img/heading_data.jpg" style="width: 100%; margin:0;" />

#

<img src="img/tab_python_client_login.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
from utils import *
import syft as sy

domain_client = sy.login(
    url=auto_detect_domain_host_ip(),
    email="info@openmined.org",
    password="changethis"
)

#

<img src="img/tab_get_dataset.png" style="width: 400px; margin:0;" />

In [ ]:
# edit MY_DATASET_URL then run this cell
MY_DATASET_URL = ""
print("My Dataset URL: ", MY_DATASET_URL)
dataset = download_dataset(MY_DATASET_URL)
dataset.head()

#

<img src="img/tab_preprocess_data.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
train, val, test = split_and_preprocess_dataset(data=dataset)

#

<img src="img/tab_make_data_private.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
def make_private_tensors(split):
    data_subjects = DataSubjectList.from_series(split["patient_ids"])
    return (
        sy.Tensor(split["images"]).private(min_val=0, max_val=255, data_subjects=data_subjects),
        sy.Tensor(split["labels"]).private(min_val=0, max_val=7, data_subjects=data_subjects)
    )

train_image_data, train_label_data = make_private_tensors(train)
val_image_data, val_label_data = make_private_tensors(val)
test_image_data, test_label_data = make_private_tensors(test)
print("Privatization of Data is complete")

#

<img src="img/tab_upload_dataset.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
domain_client.load_dataset(
    name="TissueMNIST",
    assets={
        "train_images": train_image_data,
        "train_labels": train_label_data,
        "val_images": val_image_data,
        "val_labels": val_label_data,
        "test_images": test_image_data,
        "test_labels": test_label_data,
    },
    description="This dataset is a modified form of TissueMNIST which is made available from the Broad Bioimage Benchmark Collection."
)

#

<img src="img/tab_check_dataset.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
domain_client.datasets

### 🖐 Raise your hand in Zoom and wait 

#

<img src="img/heading_network.jpg" style="width: 100%; margin:0;" />

#

<img src="img/tab_browse_networks.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
sy.networks

#

<img src="img/tab_join_network.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
network_client = sy.networks["Ad Astra Demo 1"]
domain_client.apply_to_network(network_client)

#

<img src="img/tab_see_domains.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
network_client.domains

### 🖐 Raise your hand in Zoom and wait 

#

<img src="img/heading_account.jpg" style="width: 100%; margin:0;" />

#

<img src="img/tab_create_user.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell
# ds_login_credentials = domain_client.create_user(name="Default Name", email="Default Email", password="Default Password")

data_scientist_details = {
    "name": "Sam Carter",
    "email": "sam@stargate.net",
    "password": "changethis",
    "budget": 9999,
}

# Create the data scientist user
domain_client.users.create(**data_scientist_details)

#

<img src="img/tab_copy_details.png" style="width: 400px; margin:0;" />

In [ ]:
# run this cell then copy the output
print("Please give these details to the data scientist 👇🏽")
login_details = {}
login_details["url"] = auto_detect_domain_host_ip()
login_details["name"] = data_scientist_details["name"]
login_details["email"] = data_scientist_details["email"]
login_details["password"] = data_scientist_details["password"]
print()
print(login_details)

### 🖐 Raise your hand in Zoom and wait 

<img src="img/heading_session_complete.jpg" style="width: 100%; margin:0;" />

In [ ]:
# TODO: final slide including slack details etc...